In [16]:

import asyncio
import nest_asyncio
from typing import AsyncGenerator, List, Sequence, Tuple
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, BaseChatAgent
from autogen_agentchat.base import Response
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

class UserProxyAgent(BaseChatAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name, "A human user participating in the chat.")

    @property
    def produced_message_types(self) -> List[type[ChatMessage]]:
        return [TextMessage]

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        user_input = await asyncio.get_event_loop().run_in_executor(None, input, "What number do you want to count from? ")
        print("input:",user_input)
        return Response(chat_message=TextMessage(content=user_input, source=self.name))

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        print("User Proxy Reset")

class CountDownAgent(BaseChatAgent):
    def __init__(self, name: str):
        super().__init__(name, "A simple agent that counts down.")

    @property
    def produced_message_types(self) -> Sequence[type[ChatMessage]]:
        return (TextMessage,)

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        if not messages:
            return Response(chat_message=TextMessage(content="No number received!", source=self.name))
        last_message = messages[-1].content.strip()
        if not last_message.isdigit():
            return Response(chat_message=TextMessage(content="Please enter a valid number.", source=self.name))
        count = int(last_message)
        return await self.countdown(count)

    async def countdown(self, count: int) -> Response:
        inner_messages: List[ChatMessage] = []
        for i in range(count, 0, -1):
            msg = TextMessage(content=f"{i}...", source=self.name)
            inner_messages.append(msg)
            print("msg:",msg.content)  # Print for real-time countdown
            #await asyncio.sleep(1)
        final_msg = TextMessage(content="Done!", source=self.name)
        inner_messages.append(final_msg)
        print("final:",final_msg.content)
        return Response(chat_message=final_msg, inner_messages=inner_messages)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

async def main():
    user_proxy_agent = UserProxyAgent(name="user_proxy_agent")
    countdown_agent = CountDownAgent(name="countdown_agent")
    validate_agent = CountDownAgent(name="validate_agent")
    termination = TextMentionTermination("Done!") | HandoffTermination(user_proxy_agent)
    # Create a chat group where both agents can interact
    group_chat = RoundRobinGroupChat(
        participants=[user_proxy_agent, countdown_agent, validate_agent],
        max_turns=3,  # Allow limited message exchanges to prevent infinite loops
        termination_condition=termination
    )

    # Start the chat
    stream = group_chat.run_stream(cancellation_token=CancellationToken())
    await Console(stream)
nest_asyncio.apply()
asyncio.run(main())

input: 4
msg: 4...
msg: 3...
msg: 2...
msg: 1...
final: Done!
---------- user_proxy_agent ----------
4
---------- countdown_agent ----------
4...
---------- countdown_agent ----------
3...
---------- countdown_agent ----------
2...
---------- countdown_agent ----------
1...
---------- countdown_agent ----------
Done!
---------- countdown_agent ----------
Done!
